In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [20]:
import string
import pp
import re

In [22]:
ip=pd.read_csv(r"anyfile.csv",index_col=False)

In [23]:
ip.columns

Index([u'MediaType', u'VI Sentiment', u'User Sentiment', u'Title', u'Content',
       u'PermaLink', u'Author', u'Site', u'Published Date', u'FollowersCount',
       u'FriendsCount', u'AuthorInfluence', u'ThreadClassification',
       u'Location', u'SiteRank', u'SiteReach', u'SitePageViews', u'Tags',
       u'Modified Text', u'Modified Text_stemmed', u'Modified Text_TFIDF',
       u'NounPhrases'],
      dtype='object')

In [24]:
ip=ip[['Content']]

In [32]:
stop_words=[u'i',u'want', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now', u'd', u'll', u'm', u'o', u're', u've', u'y', u'ain', u'aren', u'couldn', u'didn', u'doesn', u'hadn', u'hasn', u'haven', u'isn', u'ma', u'mightn', u'mustn', u'needn', u'shan', u'shouldn', u'wasn', u'weren', u'won', u'wouldn']
brand_words=['windows','win']
#stop_words.extend(brand_words)
replacement_patterns = [
(r'(https?[a-zA-Z0-9!$%^&*.,?:;()\\/]*)(\s|$){1,}', ''),
(r'@', 'AT'),
(r'#', 'HASHTAG')
]
patterns=[(re.compile(regex), repl) for (regex, repl) in replacement_patterns]
table=string.maketrans(string.punctuation,' '*len(string.punctuation)) #this replaces punctuation with spaces

def replace(text,patterns,table,stop_words):
    s = text.lower()
    for (pattern, repl) in patterns:
        #pattern_which_has_been_compiled.sub(replacing_string,original_string_it_has_to_change)
        s=pattern.sub(repl,s,re.I)
    s=s.translate(table)
    s = ' '.join([each for each in s.split() if each not in stop_words])
    return s

In [33]:
ip['Modified Text']=''

In [34]:
ppservers = ("*",)
job_server = pp.Server(ppservers=ppservers)
print "Starting pp with", job_server.get_ncpus(), "workers"
inputs = (ip.ix[i,"Content"].lower() for i in range(len(ip)))
jobs = [job_server.submit(replace, (each,patterns,table,stop_words), (), ("re","string")) for each in inputs]
jobs_op = [job() for job in jobs]
print "jobs done"
ip['Modified Text'] = jobs_op

Starting pp with 4 workers
C:\Anaconda2\lib\site-packages\ppworker.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  #      this list of conditions and the following disclaimer.
C:\Anaconda2\lib\site-packages\ppworker.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  #      this list of conditions and the following disclaimer.
C:\Anaconda2\lib\site-packages\ppworker.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  #      this list of conditions and the following disclaimer.
C:\Anaconda2\lib\site-packages\ppworker.py:7: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  #      this list of conditions and the following disclaimer.
jobs done


In [35]:
Cleaned_doc=pd.Series(ip["Modified Text"])

In [36]:
?TfidfVectorizer

In [37]:
vect = TfidfVectorizer(min_df=20,max_df=0.4,ngram_range=(1,2),stop_words='english',encoding=u'utf-8',decode_error= u'ignore').fit(Cleaned_doc)

In [38]:
print vect.vocabulary_.items()[0:5]
print len(vect.vocabulary_.items())

[(u'ed microsofts', 293), (u'thrilled locked', 806), (u'hashtagmicrosoft windows', 368), (u'locked', 470), (u'feature', 315)]
888


In [39]:
X_train_vectorized = vect.transform(Cleaned_doc)
Feature_TFIDF_Max_df=pd.DataFrame(X_train_vectorized.max(0).toarray()[0],vect.get_feature_names())
Feature_TFIDF_Max_df.columns=['TFIDF_Max_Scores']
Feature_TFIDF_Max_df['Features']=Feature_TFIDF_Max_df.index

In [40]:
Feature_TFIDF_Max_df_large=Feature_TFIDF_Max_df.sort_values(by=['TFIDF_Max_Scores','Features'],ascending=[False,True])

In [41]:
Feature_TFIDF_Max_df_large=Feature_TFIDF_Max_df_large[['Features','TFIDF_Max_Scores']]

In [42]:
Feature_TFIDF_Max_df_large.tail()

,Features,TFIDF_Max_Scores
outlook,outlook,0.226425
outlook ios,outlook ios,0.226425
plan outlook,plan outlook,0.226425
attimsweeneyepic update,attimsweeneyepic update,0.222155
source https,source https,0.222155


In [49]:
Feature_TFIDF_Max_df_large['Mod_Features']=Feature_TFIDF_Max_df_large['Features'].str.replace(r'\bat(\w+)',r'@\1',re.I)

In [50]:
Feature_TFIDF_Max_df_large['Mod_Features']=Feature_TFIDF_Max_df_large['Mod_Features'].str.replace(r'\bhashtag(\w+)',r'#\1',re.I)

In [51]:
Feature_TFIDF_Max_df_large.to_csv(r'TFIDF_Scored_Bigrams.csv',index=None)

In [52]:
from textblob import TextBlob as tb

In [62]:
tb(ip.ix[1,'Content'])

TextBlob("Microsoft Corporation (NASDAQ:MSFT) is working on a lighter version of Windows 10, dubbed Windows Cloud 10 or simply Windows Cloud. Reports suggest th...This text has been removed due to copyright restrictions.")

In [63]:
tb(ip.ix[1,'Content']).words

WordList(['Microsoft', 'Corporation', 'NASDAQ', 'MSFT', 'is', 'working', 'on', 'a', 'lighter', 'version', 'of', 'Windows', '10', 'dubbed', 'Windows', 'Cloud', '10', 'or', 'simply', 'Windows', 'Cloud', 'Reports', 'suggest', 'th', 'This', 'text', 'has', 'been', 'removed', 'due', 'to', 'copyright', 'restrictions'])